In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

# Arguments

In [2]:
data_dir = 'd:/fake_detection/task2/128x128/'

In [3]:
pretrained = ''
resume = ''

In [4]:
# Model
model_name = 'efficientnet-b3' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 400
start_epoch = 0
train_batch = 512
test_batch = 400
lr = 0.1
schedule = [150, 225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
val_aug = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 10.70M


In [10]:
summary(model, input_size=(3,64,64), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1            [-1, 3, 65, 65]               0
Conv2dStaticSamePadding-2           [-1, 40, 32, 32]           1,080
         GroupNorm-3           [-1, 40, 32, 32]              80
MemoryEfficientSwish-4           [-1, 40, 32, 32]               0
         ZeroPad2d-5           [-1, 40, 34, 34]               0
Conv2dStaticSamePadding-6           [-1, 40, 32, 32]             360
         GroupNorm-7           [-1, 40, 32, 32]              80
MemoryEfficientSwish-8           [-1, 40, 32, 32]               0
          Identity-9             [-1, 40, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 10, 1, 1]             410
MemoryEfficientSwish-11             [-1, 10, 1, 1]               0
         Identity-12             [-1, 10, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 40, 1, 1]             440
         I

Conv2dStaticSamePadding-121            [-1, 288, 8, 8]           7,200
       GroupNorm-122            [-1, 288, 8, 8]             576
MemoryEfficientSwish-123            [-1, 288, 8, 8]               0
        Identity-124            [-1, 288, 1, 1]               0
Conv2dStaticSamePadding-125             [-1, 12, 1, 1]           3,468
MemoryEfficientSwish-126             [-1, 12, 1, 1]               0
        Identity-127             [-1, 12, 1, 1]               0
Conv2dStaticSamePadding-128            [-1, 288, 1, 1]           3,744
        Identity-129            [-1, 288, 8, 8]               0
Conv2dStaticSamePadding-130             [-1, 48, 8, 8]          13,824
       GroupNorm-131             [-1, 48, 8, 8]              96
     MBConvBlock-132             [-1, 48, 8, 8]               0
        Identity-133             [-1, 48, 8, 8]               0
Conv2dStaticSamePadding-134            [-1, 288, 8, 8]          13,824
       GroupNorm-135            [-1, 288, 8, 8]             5

        Identity-243            [-1, 816, 1, 1]               0
Conv2dStaticSamePadding-244             [-1, 34, 1, 1]          27,778
MemoryEfficientSwish-245             [-1, 34, 1, 1]               0
        Identity-246             [-1, 34, 1, 1]               0
Conv2dStaticSamePadding-247            [-1, 816, 1, 1]          28,560
        Identity-248            [-1, 816, 4, 4]               0
Conv2dStaticSamePadding-249            [-1, 136, 4, 4]         110,976
       GroupNorm-250            [-1, 136, 4, 4]             272
     MBConvBlock-251            [-1, 136, 4, 4]               0
        Identity-252            [-1, 136, 4, 4]               0
Conv2dStaticSamePadding-253            [-1, 816, 4, 4]         110,976
       GroupNorm-254            [-1, 816, 4, 4]           1,632
MemoryEfficientSwish-255            [-1, 816, 4, 4]               0
       ZeroPad2d-256            [-1, 816, 8, 8]               0
Conv2dStaticSamePadding-257            [-1, 816, 4, 4]          20,4

        Identity-365             [-1, 58, 1, 1]               0
Conv2dStaticSamePadding-366           [-1, 1392, 1, 1]          82,128
        Identity-367           [-1, 1392, 2, 2]               0
Conv2dStaticSamePadding-368            [-1, 232, 2, 2]         322,944
       GroupNorm-369            [-1, 232, 2, 2]             464
     MBConvBlock-370            [-1, 232, 2, 2]               0
        Identity-371            [-1, 232, 2, 2]               0
Conv2dStaticSamePadding-372           [-1, 1392, 2, 2]         322,944
       GroupNorm-373           [-1, 1392, 2, 2]           2,784
MemoryEfficientSwish-374           [-1, 1392, 2, 2]               0
       ZeroPad2d-375           [-1, 1392, 6, 6]               0
Conv2dStaticSamePadding-376           [-1, 1392, 2, 2]          34,800
       GroupNorm-377           [-1, 1392, 2, 2]           2,784
MemoryEfficientSwish-378           [-1, 1392, 2, 2]               0
        Identity-379           [-1, 1392, 1, 1]               0
Conv

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in schedule:
        state['lr'] *= gamma
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
#     adjust_learning_rate(optimizer, epoch)
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 400] LR: 0.100000


RuntimeError: CUDA out of memory. Tried to allocate 102.00 MiB (GPU 0; 11.00 GiB total capacity; 8.30 GiB already allocated; 57.74 MiB free; 31.02 MiB cached)